<a href="https://colab.research.google.com/github/nrshalahudin/Project2/blob/Project/Studi_kasus_GRIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [2]:
pip install factor-Analyzer

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 1.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 202.8/202.8 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.8/98.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 43.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.9/468.9 kB 33.5 MB/s eta 0:00:00
  Created wheel for factor-Analyzer: filename=factor_analyzer-0.5.0-py2.py3-none-any.whl size=42486 sha256=dadd14dde4727acc861cbfaa0e719aa50eccd1383ba5a9f326a78c3708278be3
  Stored in directory: /root/.cache/pip/wheels/74/a2/6c/26fb1addf1ce6c60a8cef8397f2999f0a1e6e2fcddc8abf33e
Successfully built factor-Analyzer


In [3]:
df = pd.read_csv('/content/Grit - Studi Kasus.csv', delimiter=';')
df.head()

,Usia,Jenis Kelamin,Pendidikan Terakhir,Jabatan,Masa Kerja,KM1,KM2,KM3,KM4,KM5,...,AD10,AD11,AD12,AD13,Cons1,Cons2,Cons3,Cons4,Cons5,Cons6
0,26.0,Laki-laki,D4/S1,Staf / Pelaksana,6.0,4,5,3,4,2,...,3,5,1,5,7,7,6,7,6,7
1,22.0,Perempuan,D4/S1,Staf / Pelaksana,2.0,5,4,5,4,5,...,4,4,2,4,7,6,2,7,6,7
2,35.0,Laki-laki,D4/S1,NaN,5.0,2,3,3,1,5,...,1,3,4,5,7,7,3,7,3,7
3,32.0,Laki-laki,D1/D2/D3,Staf / Pelaksana,4.0,5,4,4,1,5,...,5,5,1,1,7,7,1,7,7,7
4,43.0,Laki-laki,D4/S1,Staf / Pelaksana,23.0,2,2,4,2,4,...,4,5,2,1,7,7,1,7,7,7


In [4]:
usia_modus = df['Usia'].mode()[0]
df['Usia'].fillna(usia_modus, inplace=True)

In [5]:
import pandas as pd
import numpy as np
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Memilih hanya kolom item grit (KM, PU, AD)
item_columns = df.select_dtypes(include='int64').columns.tolist()
grit_items = df[item_columns]

# Standardisasi data
scaler = StandardScaler()
grit_items_scaled = scaler.fit_transform(grit_items)

# Melakukan analisis faktor eksploratori dengan menggunakan PCA
n_factors = 5  # Jumlah faktor yang diinginkan (KM, PU, AD)
pca = PCA(n_components=n_factors)
pca.fit(grit_items_scaled)
loadings = pca.components_.T

# Menampilkan loading factor untuk setiap item
loading_factor_df = pd.DataFrame(loadings, index=item_columns, columns=[f'Factor {i}' for i in range(1, n_factors + 1)])
print("Loading Factor:")
print(loading_factor_df)

# Memilih item yang memiliki loading factor yang signifikan pada faktor-faktor utama
selected_items = loading_factor_df.abs().idxmax(axis=0)
selected_items = selected_items.to_list()
print("\nItem yang dipilih:")
print(selected_items)

# Menghitung validitas menggunakan korelasi antara item-item dan faktor-faktor utama
validitas = grit_items.corrwith(pd.DataFrame(pca.transform(grit_items_scaled), columns=loading_factor_df.columns)).mean()
print("\nHasil Validitas:", validitas)

# Menghitung reliabilitas menggunakan koefisien alpha Cronbach
def cronbach_alpha(items):
    n, k = len(items), len(items[0])
    variance_sum = sum(np.var(item) for item in items)
    total_var = np.var(np.concatenate(items))
    return (k / (k - 1)) * (1 - variance_sum / total_var)

items = grit_items[selected_items].values.T
reliabilitas = cronbach_alpha(items)
print("Hasil Reliabilitas:", reliabilitas)

Loading Factor:
       Factor 1  Factor 2  Factor 3  Factor 4  Factor 5
KM1   -0.080838  0.131415 -0.139593 -0.413002 -0.213800
KM2   -0.184630 -0.029443  0.089308 -0.355055  0.001305
KM3   -0.160447  0.155723  0.108232 -0.335207 -0.023430
KM4    0.015452  0.200980 -0.064662 -0.254942  0.082464
KM5   -0.228988  0.028191  0.225938  0.141486 -0.083955
KM6   -0.254651  0.054005  0.023477  0.028893 -0.013686
KM7    0.107836  0.200154  0.095530 -0.018634 -0.253172
KM8    0.098230  0.221734 -0.136630 -0.151781  0.134534
PU1    0.038887  0.171645 -0.060698 -0.097475 -0.202255
PU2   -0.177244  0.051556  0.105384  0.131832 -0.086881
PU3   -0.172234  0.162955 -0.008760  0.252146 -0.302670
PU4   -0.151792  0.177797 -0.263269  0.216785 -0.115794
PU5    0.134496  0.301020 -0.058279  0.049217 -0.080288
PU6    0.038070  0.258336 -0.200774 -0.204545  0.009088
PU7   -0.127654  0.208019 -0.108096  0.002947 -0.156025
PU8   -0.156440  0.137245 -0.185111  0.025577 -0.345172
PU9    0.082837  0.213165  0.028

In [6]:
df.isnull().sum()

Usia                   0
Jenis Kelamin          0
Pendidikan Terakhir    0
Jabatan                1
Masa Kerja             0
KM1                    0
KM2                    0
KM3                    0
KM4                    0
KM5                    0
KM6                    0
KM7                    0
KM8                    0
PU1                    0
PU2                    0
PU3                    0
PU4                    0
PU5                    0
PU6                    0
PU7                    0
PU8                    0
PU9                    0
PU10                   0
PU11                   0
AD1                    0
AD2                    0
AD3                    0
AD4                    0
AD5                    0
AD6                    0
AD7                    0
AD8                    0
AD9                    0
AD10                   0
AD11                   0
AD12                   0
AD13                   0
Cons1                  0
Cons2                  0
Cons3                  0


In [7]:
df['Jabatan'].unique()

array(['Staf / Pelaksana', nan, 'Manajer Lini / Pejabat Eselon IV/ III',
       'Eksekutif / Pejabat Eselon II'], dtype=object)

In [8]:
df['Jabatan'].fillna('empty', inplace=True)

In [9]:
df['Jabatan'].unique()

array(['Staf / Pelaksana', 'empty',
       'Manajer Lini / Pejabat Eselon IV/ III',
       'Eksekutif / Pejabat Eselon II'], dtype=object)

In [10]:
import pandas as pd
import numpy as np
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Memilih hanya kolom item grit (KM, PU, AD)
item_columns = df.select_dtypes(include='int64').columns.tolist()
grit_items = df[item_columns]

# Standardisasi data
scaler = StandardScaler()
grit_items_scaled = scaler.fit_transform(grit_items)

# Melakukan analisis faktor eksploratori dengan menggunakan PCA
n_factors = 5  # Jumlah faktor yang diinginkan (KM, PU, AD)
pca = PCA(n_components=n_factors)
pca.fit(grit_items_scaled)
loadings = pca.components_.T

# Menampilkan loading factor untuk setiap item
loading_factor_df = pd.DataFrame(loadings, index=item_columns, columns=[f'Factor {i}' for i in range(1, n_factors + 1)])
print("Loading Factor:")
print(loading_factor_df)

# Memilih item yang memiliki loading factor yang signifikan pada faktor-faktor utama
selected_items = loading_factor_df.abs().idxmax(axis=0)
selected_items = selected_items.to_list()
print("\nItem yang dipilih:")
print(selected_items)

# Menghitung validitas menggunakan korelasi antara item-item dan faktor-faktor utama
validitas = grit_items.corrwith(pd.DataFrame(pca.transform(grit_items_scaled), columns=loading_factor_df.columns)).mean()
print("\nHasil Validitas:", validitas)

# Menghitung reliabilitas menggunakan koefisien alpha Cronbach
def cronbach_alpha(items):
    n, k = len(items), len(items[0])
    variance_sum = sum(np.var(item) for item in items)
    total_var = np.var(np.concatenate(items))
    return (k / (k - 1)) * (1 - variance_sum / total_var)

items = grit_items[selected_items].values.T
reliabilitas = cronbach_alpha(items)
print("Hasil Reliabilitas:", reliabilitas)

Loading Factor:
       Factor 1  Factor 2  Factor 3  Factor 4  Factor 5
KM1   -0.080838  0.131415 -0.139593 -0.413002 -0.213800
KM2   -0.184630 -0.029443  0.089308 -0.355055  0.001305
KM3   -0.160447  0.155723  0.108232 -0.335207 -0.023430
KM4    0.015452  0.200980 -0.064662 -0.254942  0.082464
KM5   -0.228988  0.028191  0.225938  0.141486 -0.083955
KM6   -0.254651  0.054005  0.023477  0.028893 -0.013686
KM7    0.107836  0.200154  0.095530 -0.018634 -0.253172
KM8    0.098230  0.221734 -0.136630 -0.151781  0.134534
PU1    0.038887  0.171645 -0.060698 -0.097475 -0.202255
PU2   -0.177244  0.051556  0.105384  0.131832 -0.086881
PU3   -0.172234  0.162955 -0.008760  0.252146 -0.302670
PU4   -0.151792  0.177797 -0.263269  0.216785 -0.115794
PU5    0.134496  0.301020 -0.058279  0.049217 -0.080288
PU6    0.038070  0.258336 -0.200774 -0.204545  0.009088
PU7   -0.127654  0.208019 -0.108096  0.002947 -0.156025
PU8   -0.156440  0.137245 -0.185111  0.025577 -0.345172
PU9    0.082837  0.213165  0.028

In [11]:
numeric_columns = df.select_dtypes(include=['int']).columns

# Mengganti nilai-nilai yang lebih besar dari 5 dengan nilai 5 pada kolom-kolom numerik
df[numeric_columns] = df[numeric_columns].clip(upper=5)

In [12]:
import pandas as pd
import numpy as np
from factor_analyzer import FactorAnalyzer
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# Memilih hanya kolom item grit (KM, PU, AD)
item_columns = df.select_dtypes(include='int64').columns.tolist()
grit_items = df[item_columns]

# Standardisasi data
scaler = StandardScaler()
grit_items_scaled = scaler.fit_transform(grit_items)

# Melakukan analisis faktor eksploratori dengan menggunakan PCA
n_factors = 3  # Jumlah faktor yang diinginkan (KM, PU, AD)
pca = PCA(n_components=n_factors)
pca.fit(grit_items_scaled)
loadings = pca.components_.T

# Menampilkan loading factor untuk setiap item
loading_factor_df = pd.DataFrame(loadings, index=item_columns, columns=[f'Factor {i}' for i in range(1, n_factors + 1)])
print("Loading Factor:")
print(loading_factor_df)

# Memilih item yang memiliki loading factor yang signifikan pada faktor-faktor utama
selected_items = loading_factor_df.abs().idxmax(axis=0)
selected_items = selected_items.to_list()
print("\nItem yang dipilih:")
print(selected_items)

# Menghitung validitas menggunakan korelasi antara item-item dan faktor-faktor utama
validitas = grit_items.corrwith(pd.DataFrame(pca.transform(grit_items_scaled), columns=loading_factor_df.columns)).mean()
print("\nHasil Validitas:", validitas)

# Menghitung reliabilitas menggunakan koefisien alpha Cronbach
def cronbach_alpha(items):
    n, k = len(items), len(items[0])
    variance_sum = sum(np.var(item) for item in items)
    total_var = np.var(np.concatenate(items))
    return (k / (k - 1)) * (1 - variance_sum / total_var)

items = grit_items[selected_items].values.T
reliabilitas = cronbach_alpha(items)
print("Hasil Reliabilitas:", reliabilitas)

Loading Factor:
       Factor 1  Factor 2  Factor 3
KM1   -0.095554  0.131327  0.000875
KM2   -0.200779 -0.035823 -0.160754
KM3   -0.168920  0.148981 -0.178738
KM4    0.024657  0.205152  0.059474
KM5   -0.225484  0.029547 -0.110990
KM6   -0.272868  0.055973 -0.046510
KM7    0.109927  0.191507 -0.135571
KM8    0.095611  0.220689  0.034425
PU1    0.042040  0.170110  0.002198
PU2   -0.184932  0.048787 -0.075045
PU3   -0.180461  0.167047  0.024174
PU4   -0.162497  0.194767  0.268967
PU5    0.141066  0.297976  0.007980
PU6    0.030060  0.261722  0.076727
PU7   -0.137306  0.211396  0.035714
PU8   -0.179754  0.142080  0.063236
PU9    0.090113  0.210901 -0.028237
PU10  -0.214840  0.075374  0.054697
PU11   0.137945  0.264287 -0.117224
AD1   -0.248907  0.038960  0.051881
AD2   -0.226150  0.086501 -0.065619
AD3   -0.186651  0.072436  0.104449
AD4    0.108828  0.229520 -0.166945
AD5   -0.162335  0.090988  0.188155
AD6   -0.218570  0.060868  0.056581
AD7   -0.205437  0.100445  0.096071
AD8    0.071

In [13]:
kolom_drop = ['Cons1', 'Cons2', 'Cons3', 'Cons4', 'Cons5', 'Cons6']
df = df.drop(columns=kolom_drop)

In [14]:
# Memilih hanya kolom item grit (KM, PU, AD)
item_columns = df.select_dtypes(include='int64').columns.tolist()
grit_items = df[item_columns]

# Standardisasi data
scaler = StandardScaler()
grit_items_scaled = scaler.fit_transform(grit_items)

# Melakukan analisis faktor eksploratori dengan menggunakan PCA
n_factors = 6  # Jumlah faktor yang diinginkan (KM, PU, AD)
pca = PCA(n_components=n_factors)
pca.fit(grit_items_scaled)
loadings = pca.components_.T

# Menampilkan loading factor untuk setiap item
loading_factor_df = pd.DataFrame(loadings, index=item_columns, columns=[f'Factor {i}' for i in range(1, n_factors + 1)])
print("Loading Factor:")
print(loading_factor_df)

# Memilih item yang memiliki loading factor yang signifikan pada faktor-faktor utama
selected_items = loading_factor_df.abs().idxmax(axis=0)
selected_items = selected_items.to_list()
print("\nItem yang dipilih:")
print(selected_items)

# Menghitung validitas menggunakan korelasi antara item-item dan faktor-faktor utama
validitas = grit_items.corrwith(pd.DataFrame(pca.transform(grit_items_scaled), columns=loading_factor_df.columns)).mean()
print("\nHasil Validitas:", validitas)

# Menghitung reliabilitas menggunakan koefisien alpha Cronbach
def cronbach_alpha(items):
    n, k = len(items), len(items[0])
    variance_sum = sum(np.var(item) for item in items)
    total_var = np.var(np.concatenate(items))
    return (k / (k - 1)) * (1 - variance_sum / total_var)

items = grit_items[selected_items].values.T
reliabilitas = cronbach_alpha(items)
print("Hasil Reliabilitas:", reliabilitas)

Loading Factor:
      Factor 1  Factor 2  Factor 3  Factor 4  Factor 5  Factor 6
KM1  -0.109660  0.119408 -0.168906 -0.445927 -0.112002 -0.016063
KM2  -0.198935 -0.042400  0.134102 -0.395789  0.031912  0.233012
KM3  -0.175264  0.147736  0.138630 -0.414515  0.101232  0.115004
KM4   0.013546  0.202148 -0.161913 -0.194180 -0.098737  0.344236
KM5  -0.234688  0.025087  0.342925  0.036965 -0.080047  0.006787
KM6  -0.289831  0.042410  0.149672 -0.068643  0.085286  0.006553
KM7   0.117278  0.208793  0.106210 -0.112453 -0.264903 -0.070221
KM8   0.090040  0.221414 -0.201261 -0.106596  0.169286 -0.040741
PU1   0.043988  0.176481 -0.202269 -0.074970 -0.075777 -0.138245
PU2  -0.192347  0.045298  0.206289  0.073343 -0.215011  0.176382
PU3  -0.203169  0.156674  0.104709  0.149106 -0.316729 -0.151795
PU4  -0.208952  0.165841 -0.168241  0.191006 -0.233038 -0.000719
PU5   0.143816  0.313508 -0.177443  0.067917 -0.050493 -0.059782
PU6   0.017319  0.256404 -0.279406 -0.184676  0.114952 -0.062411
PU7  -0.1

In [15]:
selected_items = ['KM6', 'AD9', 'KM5', 'KM1', 'PU8', 'AD11']

# Memfilter DataFrame hanya untuk item-item final yang dipilih
df_selected_items = df[['Jabatan'] + selected_items]

# Membuat norma umum berdasarkan jenjang jabatan
norma_umum = df_selected_items.groupby('Jabatan').mean()

print(norma_umum)

                                            KM6       AD9       KM5       KM1  \
Jabatan                                                                         
Eksekutif / Pejabat Eselon II          4.500000  2.375000  4.250000  4.500000   
Manajer Lini / Pejabat Eselon IV/ III  4.162162  2.216216  4.162162  4.135135   
Staf / Pelaksana                       4.093567  2.760234  3.953216  4.058480   
empty                                  4.000000  5.000000  5.000000  2.000000   

                                            PU8      AD11  
Jabatan                                                    
Eksekutif / Pejabat Eselon II          4.125000  4.375000  
Manajer Lini / Pejabat Eselon IV/ III  3.945946  4.189189  
Staf / Pelaksana                       3.573099  4.023392  
empty                                  3.000000  3.000000  
